In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
################################
# Imports
################################

import pandas as pd
import h5py
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import glob
import re
from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import os
import sys
import subprocess
import random
from random import sample
from tqdm import tqdm
from functools import partial

from sklearn.metrics import confusion_matrix


import librosa

In [3]:
################################
# configs
################################

#################
# spectrogram
#################

# Mean of flute = -6.438913345336914
# Median of flute = -6.118330955505371
# stdDev of flute = 4.377377986907959
# Max of flute = 1.8442230224609375
# Min of flute = -39.0754280090332

# Mean of piano = -6.015857219696045
# Median of piano = -5.299488544464111
# stdDev of piano = 4.420877456665039
# Max of piano = 1.5825170278549194
# Min of piano = -40.520179748535156

spectrogramStats = {
                    'flute': {'mean': -6.438913345336914, 'median': -6.118330955505371, 'stdDev': 4.377377986907959, 'max': 1.8442230224609375, 'min': -39.0754280090332},
                    'piano': {'mean': -6.015857219696045, 'median': -5.299488544464111, 'stdDev': 4.420877456665039, 'max': 1.5825170278549194, 'min': -40.520179748535156}
                    }

# standardizationStyleOptions = normal, logNormal, uniform
# following needs to be 'SUBTRACTED' from the data
centerOffset = {
                'flute': {'normal': spectrogramStats['flute']['mean'], 'logNormal': spectrogramStats['flute']['mean'], 'uniform': (spectrogramStats['flute']['min'] + spectrogramStats['flute']['max'])/2},
                'piano': {'normal': spectrogramStats['piano']['mean'], 'logNormal': spectrogramStats['piano']['mean'], 'uniform': (spectrogramStats['piano']['min'] + spectrogramStats['piano']['max'])/2}
                }

# following needs to be 'DIVIDED' to the data
divFactor = {
            'flute': {'normal': 3*spectrogramStats['flute']['stdDev'], 'logNormal': (1.1*spectrogramStats['flute']['max'] - spectrogramStats['flute']['mean']) , 'uniform': 1*(spectrogramStats['flute']['max'] - spectrogramStats['flute']['min'])/2},
            'piano': {'normal': 3*spectrogramStats['piano']['stdDev'], 'logNormal': (1.1*spectrogramStats['piano']['max'] - spectrogramStats['piano']['mean']) , 'uniform': 1*(spectrogramStats['piano']['max'] - spectrogramStats['piano']['min'])/2}
            }


####################
# training params
####################
STANDARDIZATION_STYLE = 'uniform'
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 50
LEARNING_RATE = 0.0001
LAMBDA_CYCLE = 10
LAMBDA_IDENTITY = 5
GRADIENT_PENALTY = 0
ADAM_BETA1 = 0
ADAM_BETA2 = 0.9
NUM_RESIDUALS = 6

####################
# checkpoint options
####################
SAVE_CHECKPOINTS = True

####################
# find GPU device
####################
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (f'Device = {DEVICE}, # of CUDA devices = {torch.cuda.device_count()}')


###################
# add fileTag
###################
fileTag = f'lr_{LEARNING_RATE}_cyc_{LAMBDA_CYCLE}_id_{LAMBDA_IDENTITY}_b1_{ADAM_BETA1}_b2_{ADAM_BETA2}_numRes_{NUM_RESIDUALS}_gp_{GRADIENT_PENALTY}_stdStyle_{STANDARDIZATION_STYLE}'


Device = cuda:0, # of CUDA devices = 1


In [4]:
################################
# discriminator
################################

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 4, stride = stride, padding = 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=1, features=[16, 32, 64, 128], numFlatFeatures = 300):
        
        super().__init__()
        
        # output shape = 168 x 128
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 4, stride = 2, padding = 1, bias=True, padding_mode="reflect"),            
            nn.LeakyReLU(0.2)
        )
        
        
        layers = []   
        in_channels = features[0]
        
        # linear layer will have 20 x 15 features flattened
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=2))
            in_channels = feature        
        
        layers.append(nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"), 
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(in_features = numFlatFeatures, out_features = 1)
        ))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):        
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
        
    
def test():
    x = torch.randn((5,1,336,256))
    model = Discriminator(in_channels=1)
    print (model)
    preds = model(x)
    print(preds.shape)

test()

Discriminator(
  (initial): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Block(
      (conv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Block(
      (conv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, 

In [5]:
################################
# generator
################################

class EncoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False, **kwargs):
        super().__init__()
        self.encoderConv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.encoderConv(x)


class DecoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False):
        super().__init__()
        
        # nearest neighbor upsample + same convolution
        self.decoderConv = nn.Sequential(            
            nn.Upsample(scale_factor = 2, mode='nearest'),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=0),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.decoderConv(x)
    

class ResidualBlock(nn.Module):
    def __init__(self, channels, use_tanh = False):
        super().__init__()
        self.block = nn.Sequential(
            EncoderConvBlock(channels, channels, use_act = True,  use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1),
            EncoderConvBlock(channels, channels, use_act = False, use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    
    def __init__(self, img_channels, use_tanh = False, num_features = 16, num_residuals=6):
        
        super().__init__()
        
        # same convolution. output channels = 16
        self.initial = nn.Sequential(            
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )
        
        
        self.down_blocks = nn.ModuleList(
            [
                # output shape = floor((W - F + 2P) / S) + 1 . 
                # Wout x Hout = 128 x 168, for Win x Hin = 256 x 336, output channels = 32
                EncoderConvBlock(num_features, num_features*2, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
                
                # Wout x Hout = 64 x 84, for Win x Hin = 128 x 168, output channels = 64
                EncoderConvBlock(num_features*2, num_features*4, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
            ]
        )

        # same convolutions in residual blocks. Shape remains 64 x 86, numChannels = 64
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4, use_tanh = use_tanh) for _ in range(num_residuals)]
        )

        self.up_blocks = nn.ModuleList(
            [
                # Wout x Hout = 128 x 168, numChannels = 32
                DecoderConvBlock(num_features*4, num_features*2, use_act = True, use_tanh = use_tanh),
                
                # Wout x Hout = 256 x 336, numChannels = 16
                DecoderConvBlock(num_features*2, num_features, use_act = True, use_tanh = use_tanh)
            ]
        )

        # same convolution, numChannels = 1
        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        
        # same convolution
        x = self.initial(x)
        
        # down sampling
        for layer in self.down_blocks:
            x = layer(x)
            
        # same convolutions
        x = self.residual_blocks(x)
        
        # upsampling
        for layer in self.up_blocks:
            x = layer(x)
        
        # same convolution
        x = self.last(x)
        
        return torch.tanh(x)

    
def test():
    x = torch.randn((5,1,336,256))
    model = Generator(img_channels=1, num_features = 16, num_residuals=9)
    print (model)
    gen = model(x)    
    print(gen.shape)

    
test()

Generator(
  (initial): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (down_blocks): ModuleList(
    (0): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): EncoderConvBlock(
          (encoderConv): Sequential(
            (0): Conv2d(64, 64, kernel_si

torch.Size([5, 1, 336, 256])


In [6]:
# data loader

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, rootDir):
        'Initialization'
        self.rootDir = rootDir
        self.fileList = os.listdir(rootDir)        
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.fileList)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = np.load(os.path.join(self.rootDir, self.fileList[index]))
        
        return X

In [7]:
##########################
# dataset class
##########################

class PianoFluteDataset(Dataset):
    def __init__(self, root_piano, root_flute, standardizationStyle):
        self.root_piano = root_piano
        self.root_flute = root_flute

        random.seed(2)
        self.piano_images = os.listdir(root_piano)
        self.flute_images = sample(os.listdir(root_flute), len(os.listdir(root_flute)))
        
        self.length_dataset = max(len(self.piano_images), len(self.flute_images))

        self.piano_dataset_length = len(self.piano_images)
        self.flute_dataset_length = len(self.flute_images)
        
        self.standardizationStyle = standardizationStyle
        
        # default to normal for standardizing the data
        self.offsetFlute = centerOffset['flute']['normal']
        self.offsetPiano = centerOffset['piano']['normal']
        
        self.divFlute = divFactor['flute']['normal']
        self.divPiano = divFactor['piano']['normal']
        
        if standardizationStyle == 'uniform':
            print ('Using uniform assumption...')
            self.offsetFlute = centerOffset['flute']['uniform']
            self.offsetPiano = centerOffset['piano']['uniform']
        
            self.divFlute = divFactor['flute']['uniform']
            self.divPiano = divFactor['piano']['uniform']
            
        else:
            print ('Using logNormal assumption...')
            self.offsetFlute = centerOffset['flute']['logNormal']
            self.offsetPiano = centerOffset['piano']['logNormal']
        
            self.divFlute = divFactor['flute']['logNormal']
            self.divPiano = divFactor['piano']['logNormal']
        

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        
        flute_image = self.flute_images[index % self.flute_dataset_length]
        piano_image = self.piano_images[index % self.piano_dataset_length]

        flute_path = os.path.join(self.root_flute, flute_image)
        piano_path = os.path.join(self.root_piano, piano_image)
        
        flute_img = np.load(flute_path)
        piano_img = np.load(piano_path)
        
        # add extra dimension for the single channel
        flute_img = flute_img[None, :]
        piano_img = piano_img[None, :]
        
        # standardize the scale
        flute_img = (flute_img - self.offsetFlute) / self.divFlute
        piano_img = (piano_img - self.offsetPiano) / self.divPiano        

        return flute_img, piano_img

In [8]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):    
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

In [9]:
################################
# paths to directories
################################

curDir = os.getcwd()
trainSetDir = f'{curDir}/../../dataSuperSet/processedData/trainSet'
fluteTrainSetDir = f'{trainSetDir}/flute/cqtChunks'
pianoTrainSetDir = f'{trainSetDir}/piano/cqtChunks'

# create directories to store checkpoint outputs
checkPointDir = f'{curDir}/checkPoints'
checkPointModelDir = f'{checkPointDir}/models'
checkPointImageDir = f'{checkPointDir}/images'
checkPointLossTrackingDir = f'{checkPointDir}/lossTracking'

os.system(f'mkdir -p {checkPointDir}')
os.system(f'mkdir -p {checkPointModelDir}')
os.system(f'mkdir -p {checkPointImageDir}')
os.system(f'mkdir -p {checkPointLossTrackingDir}')

0

In [10]:
################################
# datasets
################################

# create dataset
dataset = PianoFluteDataset(pianoTrainSetDir, fluteTrainSetDir, STANDARDIZATION_STYLE)

# create dataloader
loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

Using uniform assumption...


In [11]:
#########################################
# instantiate networks and optimizers
#########################################

# discriminator piano
disc_P = Discriminator(in_channels=1).to(DEVICE)

# discriminator flute
disc_F = Discriminator(in_channels=1).to(DEVICE)

# generator piano
gen_P = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# generator piano
gen_F = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# optimizer discriminator
opt_disc = optim.Adam(list(disc_P.parameters()) + list(disc_F.parameters()), lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))

# optimizer generator
opt_gen  = optim.Adam(list(gen_P.parameters())  + list(gen_F.parameters()),  lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))


# Losses

# For cycle consistency and identity loss
L1 = nn.L1Loss() 

# adversarial loss
mse = nn.MSELoss()


In [12]:
################################
# training
################################

generatorLossProgression = np.zeros(NUM_EPOCHS)
discriminatorLossProgression = np.zeros(NUM_EPOCHS)
identityLossProgression = np.zeros(NUM_EPOCHS)
cycleLossProgression = np.zeros(NUM_EPOCHS)

for epoch in range(NUM_EPOCHS):    
    
    for idx, (flute, piano) in enumerate(loader):
        
        # move data to device
        piano = piano.to(DEVICE)
        flute = flute.to(DEVICE)
                
        
        ##############################
        # Discriminator training
        ##############################
        
        # piano generator output 
        fake_piano = gen_P (flute)        
        
        # piano discriminator
        D_P_real = disc_P(piano)
        D_P_fake = disc_P(fake_piano.detach())                
        
        # real = 1, fake = 0
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_real))
                
        
        # total piano discriminator loss
        Disc_piano_loss = D_P_real_loss + D_P_fake_loss

        # flute generator output
        fake_flute = gen_F(piano)
        
        # flute discriminator
        D_F_real = disc_F(flute)
        D_F_fake = disc_F(fake_flute.detach())
        
        # real = 1, fake = 0
        D_F_real_loss = mse(D_F_real, torch.ones_like(D_F_real))
        D_F_fake_loss = mse(D_F_fake, torch.zeros_like(D_F_real))
        
        # total flute discriminator loss
        Disc_flute_loss = D_F_real_loss + D_F_fake_loss

        # Overall discriminator loss
        D_loss = (Disc_flute_loss + Disc_piano_loss)/2

        # zero out the gradients
        opt_disc.zero_grad()
        
        # backprop
        D_loss.backward()
        
        # update discriminator params
        opt_disc.step()        
        
        ##############################
        # Generator training
        ##############################
        
        # Adversarial Loss for both generators
        D_P_fake = disc_P(fake_piano)
        D_F_fake = disc_F(fake_flute)
        
        # generator wants fake to be detected real
        loss_G_F = mse(D_F_fake, torch.ones_like(D_F_fake))
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))

        # Cycle Loss
        cycle_piano = gen_P(fake_flute)
        cycle_flute = gen_F(fake_piano)
        cycle_piano_loss = L1(piano, cycle_piano)
        cycle_flute_loss = L1(flute, cycle_flute)
        
        cycle_loss = cycle_flute_loss + cycle_piano_loss

        # Identity Loss
        identity_flute = gen_F(flute)
        identity_piano = gen_P(piano)
        identity_piano_loss = L1(piano, identity_piano)
        identity_flute_loss = L1(flute, identity_flute)
        
        identity_loss = identity_flute_loss + identity_piano_loss

        # Overall Generator Loss
        G_loss = loss_G_F + loss_G_P + cycle_loss*LAMBDA_CYCLE + identity_loss*LAMBDA_IDENTITY

        # zero out the gradients
        opt_gen.zero_grad()
        
        # backprop
        G_loss.backward()
        
        # update generator params
        opt_gen.step()
        
        if idx % 100 == 0:
            
            # print the current losses
            print (f'[epoch = {epoch}, idx = {idx}]:   D_loss = {D_loss.item()}, \t G_loss = {G_loss.item()}, \t identity_loss = {identity_loss.item()}, \t cycle_loss = {cycle_loss.item()}')            
            
        if idx % 1250 == 0:      
            
            # save the fake piano and flute np array of a sample            
            np.save(f'{checkPointImageDir}/originalPiano__idx_{idx}__{fileTag}.npy', np.squeeze(piano.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakeFlute__idx_{idx}__{fileTag}.npy', np.squeeze(fake_flute.detach().cpu().numpy()[0,:]))            
            np.save(f'{checkPointImageDir}/originalFlute__idx_{idx}__{fileTag}.npy', np.squeeze(flute.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakePiano__idx_{idx}__{fileTag}.npy', np.squeeze(fake_piano.detach().cpu().numpy()[0,:]))
        
        
        # aggregate loss over all batches
        discriminatorLossProgression[epoch] += D_loss.item()
        generatorLossProgression[epoch] += G_loss.item()
        identityLossProgression[epoch] += identity_loss.item()
        cycleLossProgression[epoch] += cycle_loss.item()
    
    # average loss over the length of dataset
    discriminatorLossProgression[epoch] = discriminatorLossProgression[epoch] / len(dataset)
    generatorLossProgression[epoch] = generatorLossProgression[epoch] / len(dataset)
    identityLossProgression[epoch] = identityLossProgression[epoch] / len(dataset)
    cycleLossProgression[epoch] = cycleLossProgression[epoch] / len(dataset)
    
    # save the model and current losses
    if SAVE_CHECKPOINTS:
        save_checkpoint(gen_P, opt_gen, filename = f'{checkPointModelDir}/genp__{fileTag}.pth.tar')
        save_checkpoint(gen_F, opt_gen, filename = f'{checkPointModelDir}/genf__{fileTag}.pth.tar')
        save_checkpoint(disc_P, opt_disc, filename = f'{checkPointModelDir}/criticp__{fileTag}.pth.tar')
        save_checkpoint(disc_F, opt_disc, filename = f'{checkPointModelDir}/criticf__{fileTag}.pth.tar')
    
        # save the lossProgressions
        np.save(f'{checkPointLossTrackingDir}/discriminatorLossProgression__{fileTag}.npy', discriminatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/generatorLossProgression__{fileTag}.npy', generatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/identityLossProgression__{fileTag}.npy', identityLossProgression)
        np.save(f'{checkPointLossTrackingDir}/cycleLossProgression__{fileTag}.npy', cycleLossProgression)
    
    




[epoch = 0, idx = 0]:   D_loss = 0.49945729970932007, 	 G_loss = 23.340839385986328, 	 identity_loss = 1.5110509395599365, 	 cycle_loss = 1.5074940919876099
[epoch = 0, idx = 100]:   D_loss = 0.34270986914634705, 	 G_loss = 2.7697229385375977, 	 identity_loss = 0.11111319065093994, 	 cycle_loss = 0.13338656723499298
[epoch = 0, idx = 200]:   D_loss = 0.38927918672561646, 	 G_loss = 2.517620086669922, 	 identity_loss = 0.09873934090137482, 	 cycle_loss = 0.10368727147579193
[epoch = 0, idx = 300]:   D_loss = 0.4070790708065033, 	 G_loss = 3.069159984588623, 	 identity_loss = 0.13068360090255737, 	 cycle_loss = 0.1527814269065857
[epoch = 0, idx = 400]:   D_loss = 0.40192997455596924, 	 G_loss = 2.4673519134521484, 	 identity_loss = 0.08891861140727997, 	 cycle_loss = 0.0961676836013794
[epoch = 0, idx = 500]:   D_loss = 0.5209285020828247, 	 G_loss = 2.234407901763916, 	 identity_loss = 0.1050582155585289, 	 cycle_loss = 0.11171086132526398
[epoch = 0, idx = 600]:   D_loss = 0.223209127

[epoch = 0, idx = 5200]:   D_loss = 0.3122217059135437, 	 G_loss = 2.4224016666412354, 	 identity_loss = 0.07002301514148712, 	 cycle_loss = 0.0821138247847557
[epoch = 0, idx = 5300]:   D_loss = 0.3971666991710663, 	 G_loss = 2.9658150672912598, 	 identity_loss = 0.12058931589126587, 	 cycle_loss = 0.11684684455394745
[epoch = 1, idx = 0]:   D_loss = 0.35404714941978455, 	 G_loss = 2.448108673095703, 	 identity_loss = 0.06775267422199249, 	 cycle_loss = 0.07536748051643372
[epoch = 1, idx = 100]:   D_loss = 0.5600109100341797, 	 G_loss = 2.2191593647003174, 	 identity_loss = 0.07675057649612427, 	 cycle_loss = 0.08207885921001434
[epoch = 1, idx = 200]:   D_loss = 0.3791208267211914, 	 G_loss = 2.108272075653076, 	 identity_loss = 0.0635828748345375, 	 cycle_loss = 0.07735180854797363
[epoch = 1, idx = 300]:   D_loss = 0.40549996495246887, 	 G_loss = 2.053974151611328, 	 identity_loss = 0.0647415742278099, 	 cycle_loss = 0.07719467580318451
[epoch = 1, idx = 400]:   D_loss = 0.3173981

[epoch = 1, idx = 5000]:   D_loss = 0.3237239420413971, 	 G_loss = 2.5228867530822754, 	 identity_loss = 0.07619936764240265, 	 cycle_loss = 0.08555172383785248
[epoch = 1, idx = 5100]:   D_loss = 0.4105217456817627, 	 G_loss = 2.34873366355896, 	 identity_loss = 0.055701859295368195, 	 cycle_loss = 0.07286825031042099
[epoch = 1, idx = 5200]:   D_loss = 0.4209855794906616, 	 G_loss = 2.633629560470581, 	 identity_loss = 0.06743118166923523, 	 cycle_loss = 0.08376413583755493
[epoch = 1, idx = 5300]:   D_loss = 0.20583973824977875, 	 G_loss = 2.433675765991211, 	 identity_loss = 0.07495883107185364, 	 cycle_loss = 0.0793590396642685
[epoch = 2, idx = 0]:   D_loss = 0.3741132616996765, 	 G_loss = 2.177152156829834, 	 identity_loss = 0.062073566019535065, 	 cycle_loss = 0.07231514900922775
[epoch = 2, idx = 100]:   D_loss = 0.4705950617790222, 	 G_loss = 2.045417547225952, 	 identity_loss = 0.06897766888141632, 	 cycle_loss = 0.08166508376598358
[epoch = 2, idx = 200]:   D_loss = 0.39557

[epoch = 2, idx = 4800]:   D_loss = 0.2817195653915405, 	 G_loss = 2.1623451709747314, 	 identity_loss = 0.05608023703098297, 	 cycle_loss = 0.06985139846801758
[epoch = 2, idx = 4900]:   D_loss = 0.25451433658599854, 	 G_loss = 2.710616111755371, 	 identity_loss = 0.07858765870332718, 	 cycle_loss = 0.09957155585289001
[epoch = 2, idx = 5000]:   D_loss = 0.1723996102809906, 	 G_loss = 2.5168871879577637, 	 identity_loss = 0.05275648832321167, 	 cycle_loss = 0.06951332092285156
[epoch = 2, idx = 5100]:   D_loss = 0.2409539520740509, 	 G_loss = 2.242267370223999, 	 identity_loss = 0.05614018812775612, 	 cycle_loss = 0.06739404797554016
[epoch = 2, idx = 5200]:   D_loss = 0.36521148681640625, 	 G_loss = 2.393367290496826, 	 identity_loss = 0.055765874683856964, 	 cycle_loss = 0.07335905730724335
[epoch = 2, idx = 5300]:   D_loss = 0.33002349734306335, 	 G_loss = 2.837979316711426, 	 identity_loss = 0.06250953674316406, 	 cycle_loss = 0.0781777948141098
[epoch = 3, idx = 0]:   D_loss = 0.

[epoch = 3, idx = 4500]:   D_loss = 0.26502981781959534, 	 G_loss = 2.3174493312835693, 	 identity_loss = 0.05980457365512848, 	 cycle_loss = 0.0730837881565094
[epoch = 3, idx = 4600]:   D_loss = 0.25228893756866455, 	 G_loss = 2.062812089920044, 	 identity_loss = 0.05692815035581589, 	 cycle_loss = 0.0724291130900383
[epoch = 3, idx = 4700]:   D_loss = 0.22844643890857697, 	 G_loss = 2.349090576171875, 	 identity_loss = 0.05065636336803436, 	 cycle_loss = 0.06434352695941925
[epoch = 3, idx = 4800]:   D_loss = 0.30065032839775085, 	 G_loss = 2.2347424030303955, 	 identity_loss = 0.050457388162612915, 	 cycle_loss = 0.06553072482347488
[epoch = 3, idx = 4900]:   D_loss = 0.1636200100183487, 	 G_loss = 2.7030537128448486, 	 identity_loss = 0.06337649375200272, 	 cycle_loss = 0.07073520123958588
[epoch = 3, idx = 5000]:   D_loss = 0.2114701271057129, 	 G_loss = 2.566171169281006, 	 identity_loss = 0.058279771357774734, 	 cycle_loss = 0.07495982944965363
[epoch = 3, idx = 5100]:   D_loss

...

[epoch = 45, idx = 2200]:   D_loss = 0.12228766828775406, 	 G_loss = 2.6423346996307373, 	 identity_loss = 0.03344973921775818, 	 cycle_loss = 0.04795753210783005
[epoch = 45, idx = 2300]:   D_loss = 0.09579554945230484, 	 G_loss = 2.8319876194000244, 	 identity_loss = 0.04136260598897934, 	 cycle_loss = 0.06291574239730835
[epoch = 45, idx = 2400]:   D_loss = 0.00054888182785362, 	 G_loss = 2.6602084636688232, 	 identity_loss = 0.035551778972148895, 	 cycle_loss = 0.051512084901332855
[epoch = 45, idx = 2500]:   D_loss = 0.039386168122291565, 	 G_loss = 2.6738457679748535, 	 identity_loss = 0.03764273598790169, 	 cycle_loss = 0.05348438397049904
[epoch = 45, idx = 2600]:   D_loss = 0.09175723791122437, 	 G_loss = 2.303079128265381, 	 identity_loss = 0.03762281686067581, 	 cycle_loss = 0.050653375685214996
[epoch = 45, idx = 2700]:   D_loss = 0.2526024580001831, 	 G_loss = 2.6766550540924072, 	 identity_loss = 0.03842848539352417, 	 cycle_loss = 0.05100081115961075
[epoch = 45, idx = 2

[epoch = 46, idx = 1900]:   D_loss = 0.1449737399816513, 	 G_loss = 2.663881778717041, 	 identity_loss = 0.03425493836402893, 	 cycle_loss = 0.04985513910651207
[epoch = 46, idx = 2000]:   D_loss = 0.12768034636974335, 	 G_loss = 2.4182441234588623, 	 identity_loss = 0.037594374269247055, 	 cycle_loss = 0.053135812282562256
[epoch = 46, idx = 2100]:   D_loss = 0.0074153924360871315, 	 G_loss = 2.6758201122283936, 	 identity_loss = 0.040677495300769806, 	 cycle_loss = 0.053013041615486145
[epoch = 46, idx = 2200]:   D_loss = 0.05795598775148392, 	 G_loss = 2.352755069732666, 	 identity_loss = 0.0327385812997818, 	 cycle_loss = 0.04671535640954971
[epoch = 46, idx = 2300]:   D_loss = 0.013665657490491867, 	 G_loss = 2.4968714714050293, 	 identity_loss = 0.035346511751413345, 	 cycle_loss = 0.04834587126970291
[epoch = 46, idx = 2400]:   D_loss = 0.1591402292251587, 	 G_loss = 2.400499105453491, 	 identity_loss = 0.03857351839542389, 	 cycle_loss = 0.051485203206539154
[epoch = 46, idx = 

[epoch = 47, idx = 1600]:   D_loss = 0.13932792842388153, 	 G_loss = 2.6435041427612305, 	 identity_loss = 0.03391468524932861, 	 cycle_loss = 0.047429732978343964
[epoch = 47, idx = 1700]:   D_loss = 0.029227863997220993, 	 G_loss = 2.5962412357330322, 	 identity_loss = 0.03201422095298767, 	 cycle_loss = 0.04630454629659653
[epoch = 47, idx = 1800]:   D_loss = 0.01087139081209898, 	 G_loss = 2.6099085807800293, 	 identity_loss = 0.03377293795347214, 	 cycle_loss = 0.04832443594932556
[epoch = 47, idx = 1900]:   D_loss = 0.05442296713590622, 	 G_loss = 2.7770628929138184, 	 identity_loss = 0.044295862317085266, 	 cycle_loss = 0.057066429406404495
[epoch = 47, idx = 2000]:   D_loss = 0.03827574849128723, 	 G_loss = 2.6909916400909424, 	 identity_loss = 0.035026416182518005, 	 cycle_loss = 0.0528971403837204
[epoch = 47, idx = 2100]:   D_loss = 0.0835135355591774, 	 G_loss = 2.4525673389434814, 	 identity_loss = 0.03393825516104698, 	 cycle_loss = 0.049027152359485626
[epoch = 47, idx =

[epoch = 48, idx = 1300]:   D_loss = 0.08910069614648819, 	 G_loss = 2.598698854446411, 	 identity_loss = 0.03253282606601715, 	 cycle_loss = 0.046803347766399384
[epoch = 48, idx = 1400]:   D_loss = 0.11191897094249725, 	 G_loss = 2.8988192081451416, 	 identity_loss = 0.04513533413410187, 	 cycle_loss = 0.07089052349328995
[epoch = 48, idx = 1500]:   D_loss = 0.06336486339569092, 	 G_loss = 2.4394302368164062, 	 identity_loss = 0.03370361030101776, 	 cycle_loss = 0.05012177675962448
[epoch = 48, idx = 1600]:   D_loss = 0.01280594989657402, 	 G_loss = 2.7477569580078125, 	 identity_loss = 0.04056437313556671, 	 cycle_loss = 0.05659664049744606
[epoch = 48, idx = 1700]:   D_loss = 0.017732784152030945, 	 G_loss = 2.7618417739868164, 	 identity_loss = 0.04475438967347145, 	 cycle_loss = 0.05713244527578354
[epoch = 48, idx = 1800]:   D_loss = 0.14622829854488373, 	 G_loss = 2.101909637451172, 	 identity_loss = 0.03213832527399063, 	 cycle_loss = 0.04846706986427307
[epoch = 48, idx = 190

[epoch = 49, idx = 1000]:   D_loss = 0.13414917886257172, 	 G_loss = 2.5213463306427, 	 identity_loss = 0.03733467310667038, 	 cycle_loss = 0.05058755725622177
[epoch = 49, idx = 1100]:   D_loss = 0.02994312345981598, 	 G_loss = 2.6314306259155273, 	 identity_loss = 0.031479611992836, 	 cycle_loss = 0.047679729759693146
[epoch = 49, idx = 1200]:   D_loss = 0.006341516971588135, 	 G_loss = 2.6278791427612305, 	 identity_loss = 0.0358402281999588, 	 cycle_loss = 0.05103876069188118
[epoch = 49, idx = 1300]:   D_loss = 0.006978978402912617, 	 G_loss = 2.6196115016937256, 	 identity_loss = 0.03490646183490753, 	 cycle_loss = 0.04996170103549957
[epoch = 49, idx = 1400]:   D_loss = 0.06327569484710693, 	 G_loss = 2.6880037784576416, 	 identity_loss = 0.03624767065048218, 	 cycle_loss = 0.055370986461639404
[epoch = 49, idx = 1500]:   D_loss = 0.03488972783088684, 	 G_loss = 2.609226703643799, 	 identity_loss = 0.03136608749628067, 	 cycle_loss = 0.04821215197443962
[epoch = 49, idx = 1600]: